# Create Video Outlines

This script will download YouTube video transcripts and create outlines from them.

In [1]:
channel_id = "UCzvJyXvqapvLWqoypJQItdw"

## Setup API Keys

In [2]:
import os
import openai
from dotenv import load_dotenv
from googleapiclient.discovery import build

load_dotenv()

YT_API_KEY = os.getenv("YT_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

AIR_API_KEY = os.getenv("AIR_API_KEY")
AIR_BASE = os.getenv("AIR_BASE")

openai.api_key = OPENAI_API_KEY
youtube = build('youtube', 'v3', developerKey=YT_API_KEY)

## Get List of Channel Videos

In [3]:
def get_channel_videos(channel_id=None, username=None):
    if channel_id:
        request = youtube.channels().list(part='contentDetails', id=channel_id)
    elif username:
        request = youtube.channels().list(part='contentDetails', forUsername=username)
    else:
        raise ValueError("Either channel_id or username must be provided")

    response = request.execute()

    # Extract playlist ID for uploaded videos
    playlist_id = response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    videos = []
    next_page_token = None

    # Fetch videos from the playlist
    while True:
        request = youtube.playlistItems().list(part='snippet', playlistId=playlist_id, maxResults=50, pageToken=next_page_token)
        response = request.execute()

        videos += [item['snippet']['resourceId']['videoId'] for item in response['items']]
        next_page_token = response.get('nextPageToken')

        if next_page_token is None:
            break

    return videos

video_ids = get_channel_videos(channel_id)
print(video_ids)

['odw6p7x9WH8', 'EKyy0wwkgcw', 'hgUkV6zb9-A', 'Aqz_WyXK9hQ', 'iW_3N7Wy3gY', '2cyJ1D_DQZE', 'kmJT70iH7x8', '0sZnBNaMzFU', 'tOOq3fTsm1Y', 'GwiX3aafS8Y', 'F2KUtha3vTU', 'J3LDyAqDmpg', '31Bu4qw7f6Q', 'ukanPTWDPTk', 'n4UeVkMCD4U', '3y6gJckVzSM', 'PltSt-ekLBA', '2-OZSnfWuYE', '3HcgQ7-jw2M', 'DzNOLF_8xaI']


## Download Transcripts

In [4]:
from transcript import download_transcript

from pyairtable import Table
from pyairtable.formulas import match

def upsert_row(table_name, record):
    """
    Upsert a row in an Airtable table.

    :param api_key: Airtable API key
    :param base_id: Airtable Base ID
    :param table_name: Name of the table in the base
    :param record: A dictionary containing the record data
    """
    table = Table(AIR_API_KEY, AIR_BASE, table_name)

    # Check if the record exists based on a unique field, e.g., Video ID
    formula = match({"Video ID": record["Video ID"]})
    existing_records = table.first(formula=formula)

    if existing_records:
        # If record exists, update it
        record_id = existing_records["id"]
        table.update(record_id, record)
        print(f"Record updated: {record_id}")
    else:
        # If record does not exist, create it
        new_record = table.create(record)
        print(f"New record created: {new_record['id']}")

record = {
    "Video ID": "12345",
    "Thumbnail": "http://example.com/thumbnail.jpg",
    "Title": "Sample Video",
    "Description": "This is a sample description",
    "Transcript": "This is a sample transcript",
    "SRT": "This is sample SRT content"
}

upsert_row("Videos", record)


video_links = [f"https://www.youtube.com/watch?v={id}" for id in video_ids]

for link in video_links:
	download_transcript(link)

Transcript saved as The Algorithmic BREAKTHROUGH That Terrified OpenAI 🤯
Transcript saved as The Secret Behind OpenAI's Coup
Transcript saved as The Unsettling Truth Linking Human and AI
Transcript saved as OpenAI Announces MAJOR Platform Upgrades
Transcript saved as One Simple Technique for AI Mastery
Transcript saved as Don't Get Left Behind in the AI Revolution
Transcript saved as 🧑‍🚀 bacteria astronauts
Transcript saved as 🤖 The Fourth Industrial Revolution is here and now
Transcript saved as Superintelligence Is MUCH Closer Than You Think
Transcript saved as What You NEED to Know About the AI Hype Cycle
Transcript saved as You Need to Act NOW to Thrive in the AI Revolution
Error: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=J3LDyAqDmpg! This is most likely caused by:

Subtitles are disabled for this video

If you are sure that the described cause is not responsible for this error and that a transcript should be retrievable, please create an issue 

## Create Outlines

Now we will create outlines from each transcript we downloaded.

In [1]:
def send_text_to_gpt4(text):
    """
    Sends text to GPT-4 and returns the response.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": "please create an outline from this video transcript: " + text}],
    )
    return response.choices[0].message.content

def save_outline(filename, content):
    """
    Saves the content to a file with '_outline' appended to the original filename.
    """
    outline_filename = f"{os.path.splitext(filename)[0]}_outline.txt"
    with open(outline_filename, 'w') as file:
        file.write(content)
    print(f"Outline saved to {outline_filename}")

def process_directory(directory):
    """
    Processes each .txt file in the specified directory.
    """
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            print(f"Processing file: {filename}...")
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r') as file:
                text_content = file.read()
                response = send_text_to_gpt4(text_content)
                if response:
                    print(f"Response for {filename}:\n{response}\n")
                    save_outline(os.path.join(directory, filename), response)

process_directory("agent_workspace")

Processing file: Building Blocks of Memory in the Brain.txt...
Response for Building Blocks of Memory in the Brain.txt:
Outline:

I. Introduction
   A. Definition and concept of memory
   B. Exploration of the idea of an engram
II. Brief history of memory study
   A. Richard Simmons' concept of engram
   B. Physical changes in the brain associated with memory recall
III. The biological mechanism of memory
   A. Role of nerve cells in memory storage  
   B. Examination of the changes that occur during memory encoding
IV. Memory formation process
   A. The use of behavioral tasks in studying memory
   B. Fear conditioning paradigm as a popular setup
V. Memory Visualization 
   A. Use of immediate early genes as markers for memory encoding
   B. Functionally linking these genes to observable processes
VI. Tagging of Memory
   A. Genetic modification of neurons
   B. Linking the activation of immediate early genes to a reporter process to enable memory visualization
VII. Memory Reactivatio